In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy 
from astropy.io import fits
from astropy.table import Table, Column, vstack, MaskedColumn, QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.io import ascii
import os
import seaborn as sns
print(os.environ['PATH'])
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
print(os.environ['PATH'])
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 11
color_palette = sns.color_palette(["#000000", "#849324", "#FFB30F", "#D62828", "#6EA2D2"])

/Users/cassiemetzger/opt/anaconda3/bin:/Library/TeX/texbin:/Users/cassiemetzger/.gem/ruby/3.1.3/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/lib/ruby/gems/3.1.0/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/bin:/Users/cassiemetzger/opt/anaconda3/bin:/Users/cassiemetzger/opt/anaconda3/condabin:/opt/local/bin:/opt/local/sbin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin:/Library/Apple/usr/bin
/Library/TeX/texbin:/Users/cassiemetzger/opt/anaconda3/bin:/Library/TeX/texbin:/Users/cassiemetzger/.gem/ruby/3.1.3/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/lib/ruby/gems/3.1.0/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/bin:/Users/cassiemetzger/opt/anaconda3/bin:/Users/cassiemetzger/opt/anaconda3/condabin:/opt/local/bin:/opt/local/sbin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin:/Library/Apple/usr/bin


In [652]:

sol = 2.997925E14
w1=sol/3.368
fc_w1=0.9921
F_w1 = 306.682
e_energy = 1.6022e-16 #J 
h = 6.626e-34 #Js
e_v1 = e_energy/h
w_v = 3E8/3.4E-6
cs = 3E8

#data download

#wise_download()
#erass1_download()
#data upload
allwise = load_allwise()
erass1 = load_erass1()

#matching sources 

c = SkyCoord(ra = erass1['RA']*u.degree, dec = erass1['DEC']*u.degree)
catalog = SkyCoord(ra = allwise['ra']*u.degree, dec = allwise['dec']*u.degree)
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <= 5 #we choose a 5 arcsec matching radius
matches = erass1[m]
matches["WISE idx"] = idx[m]
matches["WISE sep"] = d2d.arcsec[m]
numbers = matches["WISE idx"]
unique, counts = np.unique(numbers, return_counts=True)
duplicates = unique[counts > 1] 
if(len(duplicates) > 0):
    mask = np.array([False if i in duplicates else True for i in matches[f'WISE idx']])
    for val in duplicates: 
        idx = np.where(matches[f'WISE idx'] == val)[0]
        dis1 = matches[f'WISE sep'][idx[0]]
        dis2 = matches[f'WISE sep'][idx[1]]
        if(dis1 < dis2): 
            mask[idx[0]] = True
            mask[idx[1]] = False
        if(dis2 < dis1): 
            mask[idx[0]] = False
            mask[idx[1]] = True
        if(dis1 == dis2): 
            mask[idx[0]] = False
            mask[idx[1]] = True
    matches = matches[mask]
print(f"{len(matches)} IR-X-ray source matches found.")

#distance to best fit line

x = allwise['w2mpro'][matches['WISE idx']]- allwise['w3mpro'][matches['WISE idx']]
y = allwise['w1mpro'][matches['WISE idx']] - allwise['w2mpro'][matches['WISE idx']]
m = 0.83809562126163
b = -1.045969175938493
matches['distance'] = distance(m, b, x, y)
m = matches['distance'] < 0.21 
matches = matches[m]
print(f"{len(matches)} sources following best fit line cut.")

#xir cut 

matches['w1flux'] = F_w1/fc_w1*10**(-allwise['w1mpro'][matches['WISE idx']]/2.5)
w1errh = allwise['w1mpro'][matches['WISE idx']] - allwise['w1sigmpro'][matches['WISE idx']]
w1errl = allwise['w1mpro'][matches['WISE idx']] + allwise['w1sigmpro'][matches['WISE idx']]
err_w1_h = F_w1/fc_w1*10**(-w1errh/2.5)-matches['w1flux']
err_w1_l = F_w1/fc_w1*10**(-w1errl/2.5)-matches['w1flux']
matches['w1sigflux'] = 0.5*(np.abs(err_w1_h)+ np.abs(err_w1_l))
matches['ML_FLUX_1_JY'] = 1E+23 * matches['ML_FLUX_1'] * 1/(e_v1)
matches['Alpha IRX'] = -np.log10(matches['w1flux']/matches['ML_FLUX_1_JY'])/np.log10(w_v/e_v1)
m = matches['Alpha IRX'] <= 1
matches = matches[m]

#removing extended sources & galactic sources

print(f"{len(matches)} sources following IRX cut.")
m = matches["EXT_LIKE"] != 0 
matches['IAUNAME'] = [name[7:] for name in matches['IAUNAME']]
extended= Table([allwise['name'][matches['WISE idx'][m]], matches['IAUNAME'][m]], names = ('WISEA', 'eRASS1'))
extended.write("./CATALOGS/TABLEA2.fits", format='fits', overwrite=True)
m = matches['EXT_LIKE'] == 0 
matches = matches[m]
ra = allwise['ra'][matches['WISE idx']]
dec = allwise['dec'][matches['WISE idx']]
c = SkyCoord(ra*u.degree, dec*u.degree, frame = 'icrs')
long = c.galactic.b.degree
m = np.abs(long) > 10 
matches = matches[m]
print(f"{len(matches)} sources following galactic cut & extended source cut.")

# radio matching 

#load in SUMSS, FIRST, and NVSS catalogs 
sumss, first, nvss, RADIO_instrument_inf = load_radio_catalogs()
c = SkyCoord(ra = allwise['ra'][matches['WISE idx']]*u.degree,dec = allwise['dec'][matches['WISE idx']]*u.degree, frame = 'icrs')
#initializing radio columns 
matches['Radio flux'] =[np.nan]*len(matches)
matches['Radio freq'] = [np.nan]*len(matches)
matches['Radio catalog'] = Column(['None']*len(matches), dtype = 'U15')
#starting with SUMSS
catalog = SkyCoord(ra = sumss['RA']*u.degree, dec = sumss['DEC']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <=5 #we choose a 5 arcsec matching radius
matches['Radio flux'][m] = sumss['INT_FLUX_36_CM'][idx[m]]
matches['Radio freq'][m] = 843E6 
matches['Radio catalog'][m] = 'SUMSS'
#now FIRST 
catalog = SkyCoord(ra = first['RA']*u.degree, dec = first['DEC']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <=5
matches['Radio flux'][m] = first['FLUX_20_CM'][idx[m]]
matches['Radio freq'][m] = 1.4E9
matches['Radio catalog'][m] = 'FIRST'
#finally, NVSS 
catalog = SkyCoord(ra = nvss['RA']*u.degree, dec = nvss['DEC']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <=5
matches['Radio flux'][m] = nvss['FLUX_20_CM'][idx[m]]
matches['Radio freq'][m] = 1.4E9
matches['Radio catalog'][m] = 'NVSS'
#search RADIO master catalog for remaining sources that dont have a flux in these catalogs 
m = np.isnan(matches['Radio flux'])
idx = np.array(range(len(matches)))
idx = idx[m]
radius = 5 * u.arcsec
for i in idx: 
    coord = SkyCoord(ra= allwise['ra'][matches['WISE idx'][i]]*u.degree, dec = allwise['dec'][matches['WISE idx'][i]]*u.degree, frame = 'icrs')
    try: 
        table = Heasarc.query_region(coord, mission = "RADIO", radius = radius)
        table['SEARCH_OFFSET_'] = np.array([float(x[:5]) for x in table['SEARCH_OFFSET_']])
        if(np.min(table['SEARCH_OFFSET_']) <=0.0833333): 
            target = np.argmin(table['SEARCH_OFFSET_'])
            if(table['FLUX_20_CM'][target] != 0): 
                matches['Radio flux'][i] = table['FLUX_20_CM'][target]
                matches['Radio freq'][i] = cs/(20/100)
                matches['Radio catalog'][i] = table['DATABASE_TABLE'][target]
            if(table['FLUX_6_CM'][target] != 0 and table['FLUX_20_CM'][target] == 0):
                matches['Radio flux'][i] = table['FLUX_6_CM'][target]
                matches['Radio freq'][i] = cs/(6/100)
                matches['Radio catalog'][i] = table['DATABASE_TABLE'][target]
            if(table['FLUX_20_CM'][target] == 0 and table['FLUX_6_CM'][target] == 0):
                matches['Radio flux'][i] = table['FLUX_OTHER'][target]
                s = table['DATABASE_TABLE'][target]
                matches['Radio catalog'][i] = s.strip()
                m = RADIO_instrument_inf['Database'] == matches['Radio catalog'][i]
                matches['Radio freq'][i] = (float(RADIO_instrument_inf['Selected Frequency (MHz)'][m]) *1E6)
    except Exception as e:
        continue
m = np.isnan(matches['Radio flux'])
missing_radio = Table([allwise['name'][matches['WISE idx'][m]], matches['IAUNAME'][m]], names = ('WISEA', 'eRASS1'))
missing_radio.write("./CATALOGS/TABLEA3.fits", format='fits', overwrite=True)
m = (np.isnan(matches['Radio flux']) == False)
print(f"{len(matches[m])} sources following radio flux cut.")
matches = matches[m]
matches['Radio flux'] = [flux*1E-3 for flux in matches['Radio flux']] #mJy to Jy
matches['Alpha RIR'] = -np.log10(matches['Radio flux']/matches['w1flux'])/np.log10(matches['Radio freq']/w_v)
m = matches['Alpha RIR'] <= 0.43
print(f"{len(matches[m])} sources following alpha_rir cut.")
matches = matches[m]
#exclude TeVCAT sources 
tevcat = load_tevcat()
c = SkyCoord(ra = allwise['ra'][matches['WISE idx']]*u.degree, dec = allwise['dec'][matches['WISE idx']]*u.degree, frame = 'icrs')
catalog =SkyCoord(ra= tevcat['Simbad RA deg']*u.degree, dec = tevcat['Simbad DEC deg']*u.degree, frame = 'icrs') #we recommend using Simbad to retrieve the optical counterparts for the TeVCAT sources for better localization
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec > 5 #we want to exclude TeVCAT matches, we consider a match to be within 5 arcsec 
matches = matches[m]
print(f"{len(matches)} candidate sources found.")

#constructing final table 
thc_name = get_THC_name(allwise, matches)
thc = Table([thc_name, allwise['name'][matches['WISE idx']], matches['IAUNAME'], allwise['ra'][matches['WISE idx']], allwise['dec'][matches['WISE idx']], allwise['w1mpro'][matches['WISE idx']], allwise['w1sigmpro'][matches['WISE idx']], allwise['w2mpro'][matches['WISE idx']], allwise['w2sigmpro'][matches['WISE idx']], allwise['w3mpro'][matches['WISE idx']], allwise['w3sigmpro'][matches['WISE idx']], matches['w1flux'], matches['w1sigflux'], matches['ML_FLUX_1'], matches['ML_FLUX_ERR_1'], matches['ML_FLUX_P4'], matches['ML_FLUX_ERR_P4'], matches['Radio flux'], matches['Radio freq'], matches['Radio catalog'], matches['Alpha IRX'], matches['Alpha RIR']], names = ("THC", "WISEA", "1eRASS", "ra", "dec", "w1mpro", "w1sigmpro", "w2mpro", "w2sigmpro", "w3mpro", "w3sigmpro", "w1flux", "w1sigflux", "ML_FLUX_1", "ML_FLUX_ERR_1", "ML_FLUX_P4", "ML_FLUX_ERR_P4","Radio flux", "Radio freq", "Catalog", "Alpha IRX", "Alpha RIR"))
thc['r_mag'] = [np.nan]*len(thc)
for i in range(len(thc)): 
    c = SkyCoord(ra = thc['ra'][i]*u.degree, dec = thc['dec'][i]*u.degree, frame = 'icrs')
    try: 
        j = Gaia.cone_search_async(c, u.Quantity(5, u.arcsec))
        r = j.get_results()
        sel = np.argmin(r['dist'])
        thc['r_mag'][i] = r['phot_rp_mean_mag'][sel]
    except Exception as e:
        print(f"Error retrieving Gaia data for source {thc['ra'][i]}, {thc['dec'][i]}: {e}")
        continue
#multiwavelength analysis
fhl2, fhl3, fgl4, lac4, cgh1, bzcat, hsp3, whsp1, whsp2, c20, dt3, dt5, l25, mar25, mas13 = load_analysis_catalogs()
c = SkyCoord(ra = thc['ra']*u.degree, dec = thc['dec']*u.degree, frame = 'icrs')
#2fhl 
catalog = SkyCoord(ra = fhl2['RAJ2000']*u.degree, dec = fhl2['DEJ2000']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl2['Pos_err_95'][idx]
thc['2FHL Assoc'] = Column(['']*len(thc), dtype = 'U17')
thc['2FHL Assoc'][mask] = fhl2['Source_Name'][idx[mask]]
#3fhl
catalog = SkyCoord(ra = fhl3['RAJ2000']*u.degree, dec = fhl3['DEJ2000']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl3['Conf_95_SemiMajor'][idx]
thc['3FHL Assoc'] = Column(['']*len(thc), dtype = 'U18')
thc['3FHL Assoc'][mask] = fhl3['Source_Name'][idx[mask]]
#4fgl
catalog = SkyCoord(ra = fgl4['RAJ2000']*u.degree, dec = fgl4['DEJ2000']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fgl4['Conf_95_SemiMajor'][idx]
thc['4FGL Assoc'] = Column(['']*len(thc), dtype = 'U18')
thc['4FGL Assoc'][mask] = fgl4['Source_Name'][idx[mask]]
thc['4FGL Assoc'] = [name.replace(' ', '') for name in thc['4FGL Assoc']]
thc['4FGL Assoc'] = [name.replace('LJ', 'L J') for name in thc['4FGL Assoc']]
#4lac
catalog = SkyCoord(ra = lac4['RACdeg']*u.degree, dec = lac4['DECdeg']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
thc['4LAC Assoc'] = Column(['']*len(thc), dtype = 'U28')
thc['4LAC Assoc'][mask] = lac4['Assoc1'][idx[mask]]
#1cgh
catalog = SkyCoord(ra = cgh1['RA_1CGH']*u.degree, dec = cgh1['DEC_1CGH']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
thc['1CGH Assoc'] = Column(['']*len(thc), dtype = 'U28')
thc['1CGH Assoc'][mask] = cgh1['Counterpart_name'][idx[mask]]
#bzcat
catalog = SkyCoord(ra = bzcat['RA']*u.degree, dec = bzcat['DEC']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
thc['BZCAT Assoc'] = Column(['']*len(thc), dtype = 'U15')
thc['BZCAT OTYPE'] = Column(['']*len(thc), dtype = 'U24')
thc['BZCAT Assoc'][mask] = [name[10:] for name in bzcat['NAME'][idx[mask]]]
thc['BZCAT OTYPE'][mask] = bzcat['OBJECT_TYPE'][idx[mask]]
#3hsp
catalog = SkyCoord(ra = hsp3['RAJ2000']*u.degree, dec = hsp3['DEJ2000']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
thc['3HSP Assoc'] = Column(['']*len(thc), dtype = 'U20')
thc['3HSP Assoc'][mask] = hsp3['Name'][idx[mask]]
#whsp
catalog = SkyCoord(ra = whsp1['_RA']*u.degree, dec = whsp1['_DE']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
thc['1WHSP Assoc'] = Column(['']*len(thc), dtype = 'U15')
thc['1WHSP Assoc'][mask] = whsp1['_1WHSP'][idx[mask]]
catalog = SkyCoord(ra = whsp2['_RA']*u.degree, dec = whsp2['_DE']*u.degree, frame = 'icrs')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
thc['2WHSP Assoc'] = Column(['']*len(thc), dtype = 'U15')
thc['2WHSP Assoc'][mask] = whsp2['_2WHSPJ'][idx[mask]]
#costamante2020
thc['C20'] = Column(['N']*len(thc), dtype = 'U1')
for i in range(len(thc)): 
    target = thc['BZCAT Assoc'][i] 
    target = target.replace(" ", "")
    if(target != "None"): 
        target1 = target[0:9]
        for j in range(len(c20)): 
            c_t = c20[j]
            c_t1 = c_t[0:9]
            if(target1 == c_t1): 
                target2 = target[10:14]
                c_t2 = c_t[10:14]
                if(target2 == c_t2): 
                    thc['C20'][i] = 'Y'
#dabrusco2023
thc['D19'] = Column(['N']*len(thc), dtype = 'U1')
for i in range(len(thc)): 
    target = thc['WISEA'][i]
    target1 = target[0:10]
for j in range(len(dt3)): 
    catalog = dt3['WISE'][j]
    catalog1 = catalog[0:10]
    if(target1 == catalog1): 
        target2 = target[11:21]
        catalog2 = catalog[11:21]
        if(target2 == catalog2): 
            thc['D19'][i] = 'Y'
for k in range(len(dt5)): 
    catalog = dt5['WISE'][k]
    catalog1 = catalog[0:10]
    if(target1 == catalog1): 
        target2 = target[11:21]
        catalog2 = catalog[11:21]
        if(target2 == catalog2): 
            thc['D19'][i] = 'Y'
#láinez2025
thc['L25'] = Column(['N']*len(thc), dtype = 'U1')
for i in range(len(thc)): 
    target = thc['4FGL Assoc'][i] 
    target1 = target[5:12]
    for j in range(len(l25)): 
        l_t = l25[j]
        l_t1 = l_t[0:7]
        if(target1 == l_t1): 
            target2 =target[13:17]
            l_t2 = l_t[8:12]
            if(target2 == l_t2): 
                thc['L25'][i] = 'Y'
#marchesi2025
thc['MAR25'] = Column(['N']*len(thc), dtype = 'U1')
for i in range(len(thc)): 
    target = thc['BZCAT Assoc'][i]
    if(target != "None"):
        target = target.replace(" ", "")
        target1 = target[0:9]
        for j in range(len(mar25)): 
            catalog = mar25['ID_5BZCAT'][j]
            catalog1 = catalog[0:9]
            if(target1 == catalog1): 
                target2 = target[10:14]
                catalog2 = catalog[10:14]
                if(target2 == catalog2): 
                    thc['MAR25'][i] = 'Y'
#massaro2013
thc['MAS13'] = Column(['N']*len(thc), dtype = 'U1')
for i in range(len(thc)): 
    name = thc['THC'][i]
    if(name in mas13['WISE']): 
        thc['MAS13'][i] = 'Y'
        idx = np.where(mas13['WISE'] == name)[0][0]

Loading ALLWISE data...
ALLWISE data loaded successfully.
Loading eRASS1 data...
eRASS1 data loaded successfully.
5482 IR-X-ray source matches found.
3706 sources following best fit line cut.
218 sources following IRX cut.
177 sources following galactic cut & extended source cut.
Loading SUMSS data...
SUMSS data loaded successfully.
Loading FIRST data...
FIRST data loaded successfully.
Loading NVSS data...
NVSS data loaded successfully.
141 sources following radio flux cut.
137 sources following alpha_rir cut.
Loading TeVCat data...
121 candidate sources found.
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquer

/Users/cassiemetzger/opt/anaconda3/lib/python3.9/site-packages/astropy/table/column.py:1280: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
Loading 2FHL data...
2FHL data loaded successfully.
Loading 3FHL data...
3FHL data loaded successfully.
Loading 4FGL data...
4FGL data loaded successfully.
Loading 4LAC data...
4LAC data loaded successfully.
Loading 1CGH data...
1CGH data loaded successfully.
Loading BZCAT data...
BZCAT data loaded successfully.
Loading 3HSP data...
3HSP data loaded successfully.
Loading WHSP 1 and 2 data...
WHSP 1 and 2 data loaded successfully.
Loading C20 data...
Loading D19 Table 3 data...


D19 Table 3 data loaded successfully.
D19 Table 5 data loaded successfully.
Loading L25 data...
Loading MAR25 data...
MAR25 data loaded successfully.
Loading MAS13 data...
MAS13 data loaded successfully.


In [650]:
thc['ra'] = [float(format_sigfigs(num, 5)) for num in thc['ra']]
thc['dec'] = [float(format_sigfigs(num, 5)) for num in thc['dec']]
thc['w1mpro'] = [float(format_sigfigs(num, 3)) for num in thc['w1mpro']]
thc['w1sigmpro'] = [float(format_sigfigs(num, 3)) for num in thc['w1sigmpro']]
thc['w2mpro'] = [float(format_sigfigs(num, 3)) for num in thc['w2mpro']]
thc['w2sigmpro'] = [float(format_sigfigs(num, 3)) for num in thc['w2sigmpro']]
thc['w3mpro'] = [float(format_sigfigs(num, 3)) for num in thc['w3mpro']]
thc['w3sigmpro'] = [float(format_sigfigs(num, 3)) for num in thc['w3sigmpro']]
thc['w1flux'] = thc['w1flux'] *1E3 # convert to mJy
thc['w1flux'] = [float(format_sigfigs(num, 3)) for num in thc['w1flux']]
thc['w1sigflux'] = thc['w1sigflux'] *1E3 # convert to mJy
thc['w1sigflux'] = [float(format_sigfigs(num, 3)) for num in thc['w1sigflux']]
thc['ML_FLUX_1'] = [float(format_sigfigs(num*1E-12, 3))*1E12 for num in thc['ML_FLUX_1']]
thc['ML_FLUX_ERR_1'] = [float(format_sigfigs(num*1E-12, 3))*1E12 for num in thc['ML_FLUX_ERR_1']]
thc['ML_FLUX_P4'] = [float(format_sigfigs(num*1E-12, 3))*1E-12 for num in thc['ML_FLUX_P4']]
thc['ML_FLUX_ERR_P4'] = [float(format_sigfigs(num*1E-12, 3))*1E12 for num in thc['ML_FLUX_ERR_P4']]
thc['Radio flux'] = thc['Radio flux'] *1E3 # convert to mJy
thc['Radio flux'] = [float(format_sigfigs(num, 2)) for num in thc['Radio flux']]
thc['Alpha RIR'] = [float(format_sigfigs(num, 3)) for num in thc['Alpha RIR']]
thc['Alpha IRX'] = [float(format_sigfigs(num, 3)) for num in thc['Alpha IRX']]
thc['r_mag'] = [float(format_sigfigs(num, 3)) for num in thc['r_mag']]
thc['Radio freq'] = thc['Radio freq'] * 1E-9 # convert to GHz 
thc['Radio freq'] = [float(format_sigfigs(num, 3)) for num in thc['Radio freq']]

In [668]:
thc['ML_FLUX_ERR_1']

2.53e-13
1.11e-13
1.74e-13
1.45e-13
1.07e-13
7.17e-14
9.21e-14
2.97e-13
2.36e-13
7.53e-14
3.76e-13


In [246]:
#input eRASS1_Main data (Merloni et al. 2024)
hdu_list  = fits.open('./CATALOGS/eRASS1_Main.v1.1.fits')
erosita = Table(hdu_list[1].data)

In [288]:
hdu_list = fits.open("./CATALOGS/wise.fits")
allwise = Table(hdu_list[1].data)
hdu_list.close()

In [260]:
hdu_list = fits.open("./CATALOGS/wise.fits")
allwise = Table(hdu_list[1].data)
hdu_list.close()
init_len_wise = len(allwise)
print(f"{init_len_wise} following magnitude and color cuts.")
hdu_list = fits.open("./CATALOGS/eRASS1_Main.v1.1.fits")
erass1 = Table(hdu_list[1].data)
hdu_list.close()
init_len_erass = len(erass1)

IndexError: list index out of range

In [2]:
#input ALLWISE data (Cutri et al. 2013) AFTER it has been fed through wise_filter.sh 
print("Loading WISE data...")
wise_loaded = False  # Flag for load success

try:
    hdu_list = fits.open('./catalogs/IR/wise.fits')
    wise = Table(hdu_list[1].data)
    wise_loaded = True
    print("WISE data loaded successfully.")
except Exception as e:
    print(f"Error loading WISE data: {e}")
    wise = None

Loading WISE data...
WISE data loaded successfully.


In [5]:
def get_wise_flux(catalog): 
    sol = 2.997925E14 # variable sol; c in microns/s
    w1=sol/3.368
    w2=sol/4.618
    w3=sol/12.082

    fc_w1=0.9921
    fc_w2=0.9943
    fc_w3=0.9373

    F_w1 = 306.682 #Jy
    F_w2 = 170.663 #Jy
    F_w3 = 29.045 #Jy

    flux_w1 = F_w1/fc_w1*10**(-catalog['w1mpro']/2.5)
    flux_w2 = F_w2/fc_w2*10**(-catalog['w2mpro']/2.5)
    flux_w3 = F_w3/fc_w3*10**(-catalog['w3mpro']/2.5)

    #errors 
    w1errh = catalog['w1mpro'] - catalog['w1sigmpro']
    w2errh = catalog['w2mpro'] - catalog['w2sigmpro']
    w3errh = catalog['w3mpro'] - catalog['w3sigmpro']

    w1errl = catalog['w1mpro'] + catalog['w1sigmpro']
    w2errl = catalog['w2mpro'] + catalog['w2sigmpro']
    w3errl = catalog['w3mpro'] + catalog['w3sigmpro']

    #relative errors in Jy 

    err_w1_h = F_w1/fc_w1*10**(-w1errh/2.5)-flux_w1
    err_w2_h = F_w2/fc_w2*10**(-w2errh/2.5)-flux_w2
    err_w3_h = F_w3/fc_w3*10**(-w3errh/2.5)-flux_w3

    err_w1_l = F_w1/fc_w1*10**(-w1errl/2.5)-flux_w1
    err_w2_l = F_w2/fc_w2*10**(-w2errl/2.5)-flux_w2
    err_w3_l = F_w3/fc_w3*10**(-w3errl/2.5)-flux_w3

    err_w1 = 0.5*(np.abs(err_w1_h)+ np.abs(err_w1_l))
    err_w2 = 0.5*(np.abs(err_w2_h)+np.abs(err_w2_l))
    err_w3 = 0.5*(np.abs(err_w3_h)+np.abs(err_w3_l))

    flux = [flux_w1,flux_w2,flux_w3]
    err = [err_w1, err_w2, err_w3]
    catalog['w1flux'] = flux_w1 
    catalog['w2flux'] = flux_w2 
    catalog['w3flux'] = flux_w3
    catalog['w1sigflux'] = err_w1
    catalog['w2sigflux'] = err_w2
    catalog['w3sigflux'] = err_w3
    return catalog


In [6]:
wise = get_wise_flux(wise)

In [7]:
#assumes RA, DEC coords are given in degrees and radius is given in arcseconds
def matching(catalog1, RA1, DEC1, catalog2, RA2, DEC2, radius, matching_Name): 
    c = SkyCoord(ra = RA1*u.degree, dec = DEC1*u.degree, frame = 'fk5')
    catalog = SkyCoord(ra = RA2*u.degree, dec = DEC2*u.degree, frame = 'fk5')
    idx, d2d, d3d = c.match_to_catalog_sky(catalog)
    mask = d2d.arcsec <= radius
    matches = catalog1[mask]
    matches[f'{matching_Name} idx'] = idx[mask]
    matches[f'{matching_Name} sep'] = d2d.arcsec[mask]
    leng = len(matches)
    print(f'{leng} {matching_Name} matches found')
    print('Checking for duplicates...')
    numbers= matches[f'{matching_Name} idx']
    unique, counts = np.unique(numbers, return_counts=True)
    duplicates = unique[counts > 1] 
    print(f"Duplicates found: {len(duplicates)}")
    if(len(duplicates > 0)): 
        print(f"Removing duplicates...")
        mask = np.array([False if i in duplicates else True for i in matches[f'{matching_Name} idx']])
        for val in duplicates: 
            idx = np.where(matches[f'{matching_Name} idx'] == val)[0]
            dis1 = matches[f'{matching_Name} sep'][idx[0]]
            dis2 = matches[f'{matching_Name} sep'][idx[1]]
            if(dis1 < dis2): 
                mask[idx[0]] = True
                mask[idx[1]] = False
            if(dis2 < dis1): 
                mask[idx[0]] = False
                mask[idx[1]] = True
            if(dis1 == dis2): 
                mask[idx[0]] = False
                mask[idx[1]] = True
        matches = matches[mask]
        print(f"Removed {leng- len(matches)} duplicates \n {len(matches)} {matching_Name} matches found")
    return matches

In [8]:
matches = matching(erosita, erosita['RA'], erosita['DEC'], wise, wise['ra'], wise['dec'], 5.0, 'WISE')

5482 WISE matches found
Checking for duplicates...
Duplicates found: 0


In [9]:
#IR colors
matches['x'] = wise['w2mpro'][matches['WISE idx']] - wise['w3mpro'][matches['WISE idx']]
matches['y'] = wise['w1mpro'][matches['WISE idx']] - wise['w2mpro'][matches['WISE idx']]
matches['x_err'] = np.sqrt(wise['w2sigmpro'][matches['WISE idx']]**2 + wise['w3sigmpro'][matches['WISE idx']]**2)
matches['y_err'] = np.sqrt(wise['w1sigmpro'][matches['WISE idx']]**2 + wise['w2sigmpro'][matches['WISE idx']]**2)
wise['x'] = wise['w2mpro'] - wise['w3mpro']
wise['y'] = wise['w1mpro'] - wise['w2mpro']
wise['x_err'] = np.sqrt(wise['w2sigmpro']**2 + wise['w3sigmpro']**2)
wise['y_err'] = np.sqrt(wise['w1sigmpro']**2 + wise['w2sigmpro']**2)

In [10]:
m = 0.83809562126163
b = -1.045969175938493
leng = len(matches)
def distance (m,b, x,y): 
   return np.abs(m * x + b-y)/np.sqrt(m**2 + 1**2)
mask = distance(m,b, matches['x'], matches['y']) < 0.21
matches = matches[mask]
print(f'{leng - len(matches)} matches removed.\nNow {len(matches)} sources selected')

1776 matches removed.
Now 3706 sources selected


In [11]:
#choosing erosita 0.2-2.3 keV energy band, Eband = 1 keV 
e_energy = 1.6022e-16 #J 
h = 6.626e-34 #Js
e_v1 = e_energy/h
w_v = 3E8/3.4E-6


In [12]:
leng = len(matches)
matches['ML_FLUX_1_JY'] = 1E+23 * matches['ML_FLUX_1'] * 1/(e_v1)
matches['Alpha IRX'] = -np.log10(wise['w1flux'][matches['WISE idx']]/matches['ML_FLUX_1_JY'])/np.log10(w_v/e_v1)
m = (matches['Alpha IRX'] <= 1)
matches = matches[m]
print(f'{leng - len(matches)} matches removed.\nNow {len(matches)} sources selected')

3488 matches removed.
Now 218 sources selected


In [13]:
#producing Table A2 
m = matches['EXT_LIKE'] != 0 
sel = matches[m]
output_dir = "catalogs/latex_outputs"
output_file = os.path.join(output_dir, "table_A2.txt")
with open(output_file, 'w') as f: 
    for i in range(len(sel)): 
        erass_name = sel['IAUNAME'][i]
        erass_name = erass_name[7:23]
        wise_name = wise['name'][sel['WISE idx'][i]]
        f.write(f"{wise_name} & {erass_name} \\\\ \n")
print(f"Output written to {output_file}")

Output written to catalogs/latex_outputs/table_A2.txt


In [14]:
leng = len(matches)
m = matches['EXT_LIKE'] == 0
matches = matches[m]
print(f'{leng - len(matches)} matches removed.\nNow {len(matches)} sources selected')

31 matches removed.
Now 187 sources selected


In [15]:
leng = len(matches)
ra = wise['ra'][matches['WISE idx']]
dec = wise['dec'][matches['WISE idx']]
c = SkyCoord(ra*u.degree, dec*u.degree, frame = 'icrs')
lat = c.galactic.l.degree
long = c.galactic.b.degree
idx = []
for i in range(len(matches)): 
    if(long[i] < -10 or long[i] > 10): 
        idx.append(i)
matches = matches[idx]
print(f'{leng - len(matches)} matches removed.\nNow {len(matches)} sources selected')


10 matches removed.
Now 177 sources selected


In [16]:
def matching_radio(catalog1, RA1, DEC1, catalog2, RA2, DEC2, flux2, freq, radius, matching_Name): 
    c =SkyCoord(ra = RA1*u.degree, dec = DEC1*u.degree, frame = 'fk5')
    catalog = SkyCoord(ra = RA2*u.degree, dec = DEC2*u.degree, frame = 'fk5')
    idx, d2d, d3d = c.match_to_catalog_sky(catalog)
    mask = d2d.arcsec <= radius
    catalog1['Radio flux'][mask] = flux2[idx[mask]]
    catalog1['Radio flux'][mask] = catalog1['Radio flux'][mask] * 1E-3 #convert to Jy
    catalog1['Radio freq'][mask] = freq
    catalog1['Radio catalog'][mask] = matching_Name
    return catalog1

In [368]:
thc['Radio flux'] = [np.nan]*len(thc)
thc['Radio freq'] = [np.nan]*len(thc)
thc['Radio catalog'] = Column(['None']*len(thc), dtype = 'U15')
hdu_list = fits.open("./CATALOGS/radio/SUMSS.fits")
sumss = Table(hdu_list[1].data)
hdu_list.close()
c = SkyCoord(ra = thc['ra']*u.degree, dec = thc['dec']*u.degree, frame = 'icrs')
catalog = SkyCoord(ra =sumss['RA']*u.degree, dec = sumss['DEC']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <= 5 
thc['Radio flux'][m] = [float(flux)*1E-3 for flux in sumss['INT_FLUX_36_CM'][idx[m]]]
thc['Radio freq'][m] = 843e6
thc['Radio catalog'][m] = 'SUMSS'

In [371]:
m = np.isnan(thc['Radio flux'])
idx = np.array(range(len(thc)))
idx = idx[m]
radio_frequencies = ascii.read('./CATALOGS/radio/radiocatalog_frequencies.txt', delimiter = ",")

In [ ]:
#initializing radio data columns
matches['Radio flux'] = [np.nan]*len(matches)
matches['Radio freq'] = [np.nan]*len(matches)
matches['Radio catalog'] = Column(['None']*len(matches), dtype = 'U15')
#radio matching
#change paths to match your local directory structure

#SUMSS (Mauch et al. 2003)
hdu_list = fits.open('./catalogs/radio/sumss.fits')
sumss = Table(hdu_list[1].data)
matches = matching_radio(matches, wise['ra'][matches['WISE idx']], wise['dec'][matches['WISE idx']], sumss, sumss['RA'], sumss['DEC'], sumss['INT_FLUX_36_CM'], 843E6, 5.0, 'SUMSS')
#FIRST (White et al. 1997; Helfand et al. 2015)
hdu_list = fits.open('./catalogs/radio/first.fits')
first = Table(hdu_list[1].data)
matches = matching_radio(matches, wise['ra'][matches['WISE idx']], wise['dec'][matches['WISE idx']], first, first['RA'], first['DEC'], first['FLUX_20_CM'], 1.4E9, 5.0, 'FIRST')
#NVSS (Cordon et al. 1998)
hdu_list = fits.open('./catalogs/radio/nvss.fits')  
nvss = Table(hdu_list[1].data)
matches = matching_radio(matches, wise['ra'][matches['WISE idx']], wise['dec'][matches['WISE idx']], nvss, nvss['RA'], nvss['DEC'], nvss['FLUX_20_CM'], 1.4E9, 5.0, 'NVSS')
m = (np.isnan(matches['Radio flux']))
idx = np.array(range(len(matches)))
idx = idx[m]
#searching RADIO Master Catalog 
import astroquery 
from astroquery.heasarc import Heasarc
flux = matches['Radio flux']
freq = matches['Radio freq']
catalog = matches['Radio catalog']
c = 3E8
for i in range(len(idx)): 
    coord = SkyCoord(ra = wise['ra'][matches['WISE idx'][idx[i]]]*u.degree, dec = wise['dec'][matches['WISE idx'][idx[i]]]*u.degree, frame = 'fk5')
    radius = 10 *u.arcmin
    table = Heasarc.query_region(coord, mission = "RADIO", radius = radius) 
    table['SEARCH_OFFSET_'] = np.array([float(x[:5]) for x in table['SEARCH_OFFSET_']])
    if(np.min(table['SEARCH_OFFSET_']) <= 0.0833333): 
        #print(f'{idx[i]}: {wise["ra"][matches["WISE idx"][idx[i]]]}, {wise["dec"][matches["WISE idx"][idx[i]]]};')
        target = np.argmin(table['SEARCH_OFFSET_'])
        if(table['FLUX_20_CM'][target] == 0 and table['FLUX_6_CM'][target] == 0 and table['FLUX_OTHER'][target] == 0):
            catalog[idx[i]] = 'None'
            flux[idx[i]] = np.nan
            freq[idx[i]] = np.nan
        if(table['FLUX_20_CM'][target] != 0): 
            flux[idx[i]] = table['FLUX_20_CM'][target]*1E-3
            freq[idx[i]] = c/(20/100)
            catalog[idx[i]] = table['DATABASE_TABLE'][target]
        if(table['FLUX_20_CM'][target] == 0 and table['FLUX_6_CM'][target] != 0): 
            flux[idx[i]] = table['FLUX_6_CM'][target]*1E-3
            freq[idx[i]] = c/(6/100)
            catalog[idx[i]] = table['DATABASE_TABLE'][target]
        if(table['FLUX_20_CM'][target] == 0 and table['FLUX_6_CM'][target] == 0):
            print(f'Missing info on: {idx[i]}: {wise["ra"][matches["WISE idx"][idx[i]]]}, {wise["dec"][matches["WISE idx"][idx[i]]]};')

119: 19.4639509, -54.9221249;


In [ ]:
#hardcoded in from Heasarc
flux[119] =  7.04E-3 
freq[119] = 150E9 
catalog[119] = 'SPTSZSPSC'

matches['Radio flux'] = flux
matches['Radio freq'] = freq
matches['Catalog'] = catalog

In [343]:
catalog = ascii.read('./CATALOGS/radio/radiocatalog_frequencies.txt', delimiter = ",")

In [347]:
catalog['Database'][0]

'ACTEGSRCAT'

In [ ]:
#producing Table A3 
output_dir = "catalogs/latex_outputs"
output_file = os.path.join(output_dir, "table_A3.txt")
with open(output_file, 'w') as f:
    m =  (np.isnan(matches['Radio flux']) == True)
    sel = matches[m]
    for i in range(len(sel)):
        erass_name = sel['IAUNAME'][i]
        erass_name = erass_name[7:23]
        wise_name = wise['name'][sel['WISE idx'][i]]
        f.write(f"{wise_name} & {erass_name} \\\\ \n")
#change paths to match your local directory structure
print(f"Output written to {output_file}")

Output written to catalogs/latex_outputs/table_A3.txt


In [20]:
m = (np.isnan(matches['Radio flux']) == False)
leng = len(matches)
matches = matches[m]
print(f'{leng - len(matches)} matches removed.\nNow {len(matches)} sources selected')

36 matches removed.
Now 141 sources selected


In [21]:
matches['Alpha RIR'] = -np.log10(matches['Radio flux']/wise['w1flux'][matches['WISE idx']])/np.log10(matches['Radio freq']/w_v)
m = matches['Alpha RIR'] <= 0.43
leng = len(matches)
matches = matches[m]
print(f'{leng - len(matches)} matches removed.\nNow {len(matches)} sources selected')

4 matches removed.
Now 137 sources selected


In [ ]:
#removing sources that are already listed in TeVCAT (https://tevcat.org/)

#change path to match where your TeVCAT HBL list is stored 
tevcat = ascii.read('./catalogs/other_blazar-hbl_cats/HBL_newlist.csv')
c = SkyCoord(ra = wise['ra'][matches['WISE idx']]*u.degree, dec = wise['dec'][matches['WISE idx']]*u.degree, frame = 'fk5')
catalog = SkyCoord(ra = tevcat['Simbad RA deg']*u.degree, dec = tevcat['Simbad DEC deg']*u.degree, frame = 'fk5')   
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec > 5.0
leng = len(matches)
matches = matches[mask]
print(f'{leng - len(matches)} matches removed.\nNow {len(matches)} sources selected')

16 matches removed.
Now 121 sources selected


In [ ]:
#constructing final table 
ra = wise['ra'][matches['WISE idx']]
dec = wise['dec'][matches['WISE idx']]
c = SkyCoord(ra = ra*u.degree, dec = dec*u.degree, frame='fk5')
strings = c.to_string('hmsdms')
for i in range(len(strings)): 
    time_part, declination_part = strings[i].split()
    hours = time_part[0:2]
    minutes = time_part[3:5]
    degrees = declination_part[1:3]
    sign = declination_part[0]
    mins = declination_part[4:6]
    convert = float(mins)/60 
    mins = str(convert) 
    mins = mins[2:3]
    strings[i] = f'J{hours}{minutes}{sign}{degrees}{mins}'
selected = Table([strings, wise['name'][matches['WISE idx']], matches['IAUNAME'], wise['ra'][matches['WISE idx']], wise['dec'][matches['WISE idx']], wise['w1mpro'][matches['WISE idx']], wise['w1sigmpro'][matches['WISE idx']], wise['w2mpro'][matches['WISE idx']], wise['w2sigmpro'][matches['WISE idx']], wise['w3mpro'][matches['WISE idx']], wise['w3sigmpro'][matches['WISE idx']], wise['w1flux'][matches['WISE idx']], wise['w1sigflux'][matches['WISE idx']], matches['ML_FLUX_1'], matches['ML_FLUX_ERR_1'], matches['ML_FLUX_P4'], matches['ML_FLUX_ERR_P4'], matches['Radio flux'], matches['Radio freq'], matches['Catalog'], matches['Alpha IRX'], matches['Alpha RIR']])

In [24]:
from astroquery.gaia import Gaia
r_mag = []
for i in range(len(selected)): 
    c = SkyCoord(ra = selected['ra'][i]*u.degree, dec = selected['dec'][i]*u.degree, frame = 'icrs')
    j = Gaia.cone_search_async(c, u.Quantity(5, u.arcsec))
    r = j.get_results()
    sel = np.argmin(r['dist'])
    r_mag.append(r['phot_rp_mean_mag'][sel])


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [25]:
selected['r_mag'] = r_mag
m = [type(num) != np.float64 for num in selected['r_mag']]
selected['r_mag'][m] = np.nan

In [ ]:
#gamma-ray catalog matching; again, change the paths to match your local directory structure

#2FHL (Ackerman et al. 2016)
hdu_list = fits.open('./catalogs/gammaray/2FHL.fits')
fhl2 = Table(hdu_list[1].data)
c = SkyCoord(ra = selected['ra']*u.degree, dec = selected['dec']*u.degree, frame = 'fk5')
catalog = SkyCoord(ra = fhl2['RAJ2000']*u.degree, dec = fhl2['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl2['Pos_err_95'][idx]
selected['2FHL Assoc'] = Column(['']*len(selected), dtype = 'U17')
selected['2FHL Assoc'][mask] = fhl2['Source_Name'][idx[mask]]
#3FHL (Ajello et al. 2017)
hdu_list = fits.open('./catalogs/gammaray/3FHL.fits')
fhl3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fhl3['RAJ2000']*u.degree, dec = fhl3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl3['Conf_95_SemiMajor'][idx]
selected['3FHL Assoc'] = Column(['']*len(selected), dtype = 'U18')
selected['3FHL Assoc'][mask] = fhl3['Source_Name'][idx[mask]]
#4FGL_DR4 (Ballet et al. 2024)
hdu_list = fits.open('./catalogs/gammaray/4FGL_DR4.fits')
fgl4 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fgl4['RAJ2000']*u.degree, dec = fgl4['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fgl4['Conf_95_SemiMajor'][idx]
selected['4FGL Assoc'] = Column(['']*len(selected), dtype = 'U18')
selected['4FGL Assoc'][mask] = fgl4['Source_Name'][idx[mask]]
selected['4FGL Assoc'] = [name.replace(' ', '') for name in selected['4FGL Assoc']]
selected['4FGL Assoc'] = [name.replace('LJ', 'L J') for name in selected['4FGL Assoc']]
#4LAC_DR3 (Ajello et al. 2023)
hdu_list = fits.open('./catalogs/gammaray/4LAC-DR3.fits')
lac = Table(hdu_list[1].data)
catalog = SkyCoord(ra = lac['RACdeg']*u.degree, dec = lac['DECdeg']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
selected['4LAC Assoc'] = Column(['']*len(selected), dtype = 'U28')
selected['4LAC Assoc'][mask] = lac['Assoc1'][idx[mask]]
#1CGH (Arsioli et al. 2025)
hdu_list = fits.open('./catalogs/gammaray/1CGH_Preliminary_V2.0.fits')
cgh = Table(hdu_list[1].data)
catalog = SkyCoord(ra = cgh['RA_1CGH']*u.degree, dec = cgh['DEC_1CGH']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
selected['1CGH Assoc'] = Column(['']*len(selected), dtype = 'U28')
selected['1CGH Assoc'][mask] = cgh['Counterpart_name'][idx[mask]]


In [ ]:
#multi-frequency catalog matching; again, change the paths to match your local directory structure

#BZCAT (Massaro et al. 2015)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/bzcat.fits')
bzcat = Table(hdu_list[1].data)
catalog = SkyCoord(ra = bzcat['RA']*u.degree, dec = bzcat['DEC']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
selected['BZCAT Assoc'] = Column(['']*len(selected), dtype = 'U15')
selected['BZCAT OTYPE'] = Column(['']*len(selected), dtype = 'U24')
for i in range(len(selected)): 
    if(mask[i]): 
        name = bzcat['NAME'][idx[i]]
        name = name[10:]
        selected['BZCAT Assoc'][i] = name
        selected['BZCAT OTYPE'][i] = bzcat['OBJECT_TYPE'][idx[i]]
        selected['BZCAT OTYPE'][i] = selected['BZCAT OTYPE'][i].replace('  ', '')
#3HSP (Chang et al. 2019)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/3HSP.fits')
hsp3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = hsp3['RAJ2000']*u.degree, dec = hsp3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
selected['3HSP Assoc'] = Column(['']*len(selected), dtype = 'U20')
selected['3HSP Assoc'][mask] = hsp3['Name'][idx[mask]]
#1WHSP (Arsioli et al. 2015)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/1whsp.fits')
whsp1 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp1['_RA']*u.degree, dec = whsp1['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
selected['1WHSP Assoc'] = Column(['']*len(selected), dtype = 'U15')
selected['1WHSP Assoc'][mask] = whsp1['_1WHSP'][idx[mask]]
#2WHSP (Chang et al. 2017)
hdu_list= fits.open('./catalogs/other_blazar-hbl_cats/whsp2.fits')
whsp2 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp2['_RA']*u.degree, dec = whsp2['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
selected['2WHSP Assoc'] = Column(['']*len(selected), dtype = 'U15')
selected['2WHSP Assoc'][mask] = whsp2['_2WHSPJ'][idx[mask]]

In [ ]:
#TeV-peaked candidate BL Lac objects (Costamante 2020)
C20 = ascii.read('./catalogs/other_blazar-hbl_cats/C20.txt')
C20 = C20[0][:]

In [ ]:
selected['C20'] = Column(['N']*len(selected), dtype = 'U1')
for i in range(len(selected)): 
    target = selected['BZCAT Assoc'][i] 
    target = target.replace(" ", "")
    if(target != "None"): 
        target1 = target[0:9]
        for j in range(len(C20)): 
            c_t = C20[j]
            c_t1 = c_t[0:9]
            if(target1 == c_t1): 
                target2 = target[10:14]
                c_t2 = c_t[10:14]
                if(target2 == c_t2): 
                    selected['C20'][i] = 'Y'
#Two new catalogs of blazar candidates in the WISE Infrared sky (D'Abrusco et al. 2019)
#T3
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot3.fits')
dabruscot3 = Table(hdu_list[1].data)
#T5
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot5.fits')
dabruscot5 = Table(hdu_list[1].data)
selected['D19'] = Column(['N']*len(selected), dtype = 'U1')
for i in range(len(selected)): 
    target = selected['name'][i]
    target1 = target[0:10]
    for j in range(len(dabruscot3)): 
        catalog = dabruscot3['WISE'][j]
        catalog1 = catalog[0:10]
        if(target1 == catalog1): 
           target2 = target[11:21]
           catalog2 = catalog[11:21]
           if(target2 == catalog2): 
               selected['D19'][i] = 'Y'
    for k in range(len(dabruscot5)): 
        catalog = dabruscot5['WISE'][k]
        catalog1 = catalog[0:10]
        if(target1 == catalog1): 
            target2 = target[11:21]
            catalog2 = catalog[11:21]
            if(target2 == catalog2): 
                selected['D19'][i] = 'Y'

In [ ]:
#Exploring the Most Extreme Gamma-Ray Blazars Using Broadband Spectral Energy Distributions (Láinez et al. 2025)
L25 = ascii.read('./catalogs/other_blazar-hbl_cats/L25.txt')
L25 = L25[0][:]


In [ ]:
selected['L25'] = Column(['N']*len(selected), dtype = 'U1')
for i in range(len(selected)): 
    target = selected['4FGL Assoc'][i] 
    target1 = target[5:12]
    for j in range(len(L25)): 
        l_t = L25[j]
        l_t1 = l_t[0:7]
        if(target1 == l_t1): 
            target2 =target[13:17]
            l_t2 = l_t[8:12]
            if(target2 == l_t2): 
                selected['L25'][i] = 'Y'
#A new look at the extragalactic very high energy sky: Searching for TeV-emitting candidates among the X-ray-bright, non-Fermi-detected blazar population (Marchesi et al. 2025)               
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/Marchesi_Xray_Bl_AA_25.fits')
marchesi = Table(hdu_list[1].data)
selected['MAR25'] = Column(['N']*len(selected), dtype = 'U1')
for i in range(len(selected)): 
    target = selected['BZCAT Assoc'][i]
    if(target != "None"):
        target = target.replace(" ", "")
        target1 = target[0:9]
        for j in range(len(marchesi)): 
            catalog = marchesi['ID_5BZCAT'][j]
            catalog1 = catalog[0:9]
            if(target1 == catalog1): 
                target2 = target[10:14]
                catalog2 = catalog[10:14]
                if(target2 == catalog2): 
                    selected['MAR25'][i] = 'Y'
#BL Lac candidates for TeV observations (Massaro et al. 2013)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/massaro_selection.fits')
massaro = Table(hdu_list[1].data)
selected['MAS13'] = Column(['N']*len(selected), dtype = 'U1')
for i in range(len(selected)): 
    name = selected['name'][i]
    if(name in massaro['WISE']): 
        selected['MAS13'][i] = 'Y'
        idx = np.where(massaro['WISE'] == name)[0][0]


In [ ]:
#formatting for paper 
fhl2 = Column([''] *len(selected), dtype = 'U1')
m = selected['2FHL Assoc'] != ''
fhl2[m] = 'x'
fhl3 = Column([''] *len(selected), dtype = 'U1')
m = selected['3FHL Assoc'] != ''
fhl3[m] = 'x'
fgl4 = Column([''] *len(selected), dtype = 'U1')
m = selected['4FGL Assoc'] != ''
fgl4[m] = 'x'
lac = Column([''] *len(selected), dtype = 'U1')
m = selected['4LAC Assoc'] != ''
lac[m] = 'x'
cgh = Column([''] *len(selected), dtype = 'U1')
m = selected['1CGH Assoc'] != ''
cgh[m] = 'x'
bz = Column([''] *len(selected), dtype = 'U1')
m = selected['BZCAT Assoc'] != ''
bz[m] = 'x'
hs = Column([''] *len(selected), dtype = 'U1')
m = selected['3HSP Assoc'] != ''
hs[m] = 'x'
whsp2 = Column([''] *len(selected), dtype = 'U1')
m = selected['2WHSP Assoc'] != ''
whsp2[m] = 'x'
c = Column([''] *len(selected), dtype = 'U1')
m = selected['C20'] != 'N'
c[m] = 'x'
d = Column([''] *len(selected), dtype = 'U1')
m = selected['D19'] != 'N'
d[m] = 'x'
l25 = Column([''] *len(selected), dtype = 'U1')
m = selected['L25'] != 'N'
l25[m] = 'x'
mar = Column([''] *len(selected), dtype = 'U1')
m = selected['MAR25'] != 'N'
mar[m] = 'x'
mas = Column([''] *len(selected), dtype = 'U1')
m = selected['MAS13'] != 'N'
mas[m] = 'x'

In [33]:
selected.rename_column('col0', 'THC')
selected.rename_column('name', "WISEA")
selected.rename_column('IAUNAME', '1eRASS')
selected['1eRASS'] = [name[7:] for name in selected['1eRASS']]
selected.sort('ra')

In [34]:
import math
from decimal import Decimal, ROUND_HALF_UP, getcontext

def format_sigfigs(num, sigfig):
    if num is None or (isinstance(num, float) and math.isnan(num)):
        return "nan"

    # Ensure context precision is high enough
    getcontext().prec = 10

    # Convert to Decimal for consistent rounding
    d = Decimal(str(num))
    rounded = d.quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)

    # Format with exactly 2 digits after the decimal point
    return f"{rounded:.{sigfig}f}"


In [36]:
output_dir = "./catalogs/latex_outputs"
output_file = os.path.join(output_dir, "Table1.txt")
with open(output_file, 'w') as f: 
    for i in range(len(selected)): 
        f.write(f"{selected['THC'][i]} & {selected['WISEA'][i]} & {selected['1eRASS'][i]} & {format_sigfigs(selected['w1flux'][i]*1E3, 2)} $\pm$ {format_sigfigs(selected['w1sigflux'][i] *1E3, 2)} & {format_sigfigs(selected['ML_FLUX_1'][i] *1E12, 2)} $\pm$ {format_sigfigs(selected['ML_FLUX_ERR_1'][i] *1E12, 2)} & {format_sigfigs(selected['ML_FLUX_P4'][i] *1E12, 2)} $\pm$ {format_sigfigs(selected['ML_FLUX_ERR_P4'][i] *1E12, 2)} & {format_sigfigs(selected['Radio flux'][i]*1E3, 2)}  & {format_sigfigs(selected['r_mag'][i], 2)} & {format_sigfigs(selected['Alpha RIR'][i], 2)} & {format_sigfigs(selected['Alpha IRX'][i], 2)} & {fhl2[i]} & {fhl3[i]} & {fgl4[i]} & {lac[i]} & {cgh[i]} & {bz[i]} & {hs[i]} & {whsp2[i]} & {c[i]} & {d[i]} & {l25[i]} & {mar[i]} & {mas[i]} \\\\ \n")
print(f"Output written to {output_file}")

Output written to ./catalogs/latex_outputs/Table1.txt


In [ ]:
selected['ra'] = [float(format_sigfigs(num, 5)) for num in selected['ra']]
selected['dec'] = [float(format_sigfigs(num, 5)) for num in selected['dec']]
selected['w1mpro'] = [float(format_sigfigs(num, 3)) for num in selected['w1mpro']]
selected['w1sigmpro'] = [float(format_sigfigs(num, 3)) for num in selected['w1sigmpro']]
selected['w2mpro'] = [float(format_sigfigs(num, 3)) for num in selected['w2mpro']]
selected['w2sigmpro'] = [float(format_sigfigs(num, 3)) for num in selected['w2sigmpro']]
selected['w3mpro'] = [float(format_sigfigs(num, 3)) for num in selected['w3mpro']]
selected['w3sigmpro'] = [float(format_sigfigs(num, 3)) for num in selected['w3sigmpro']]
selected['w1flux'] = selected['w1flux'] *1E3 # convert to mJy
selected['w1flux'] = [float(format_sigfigs(num, 3)) for num in selected['w1flux']]
selected['w1sigflux'] = selected['w1sigflux'] *1E3 # convert to mJy
selected['w1sigflux'] = [float(format_sigfigs(num, 3)) for num in selected['w1sigflux']]
selected['ML_FLUX_1'] = [float(format_sigfigs(num, 3)) for num in selected['ML_FLUX_1']]
selected['ML_FLUX_ERR_1'] = [float(format_sigfigs(num, 3)) for num in selected['ML_FLUX_ERR_1']]
selected['ML_FLUX_P4'] = [float(format_sigfigs(num, 3)) for num in selected['ML_FLUX_P4']]
selected['ML_FLUX_ERR_P4'] = [float(format_sigfigs(num, 3)) for num in selected['ML_FLUX_ERR_P4']]
selected['Radio flux'] = selected['Radio flux'] *1E3 # convert to mJy
selected['Radio flux'] = [float(format_sigfigs(num, 2)) for num in selected['Radio flux']]


TypeError: list indices must be integers or slices, not str

In [46]:
selected['Alpha RIR'] = [float(format_sigfigs(num, 3)) for num in selected['Alpha RIR']]
selected['Alpha IRX'] = [float(format_sigfigs(num, 3)) for num in selected['Alpha IRX']]
selected['r_mag'] = [float(format_sigfigs(num, 3)) for num in selected['r_mag']]
selected['Radio freq'] = selected['Radio freq'] * 1E-9 # convert to GHz 
selected['Radio freq'] = [float(format_sigfigs(num, 3)) for num in selected['Radio freq']]

In [47]:
selected.sort('ra')

In [49]:
df = selected.to_pandas()
df.to_excel('./catalogs/THC_catalog.xlsx', index = False)
print('Catalog saved as THC_catalog.xlsx')

Catalog saved as THC_catalog.xlsx


In [51]:
tbl = QTable(selected)
tbl['ra'] = tbl['ra']*u.degree
tbl['dec'] = tbl['dec']*u.degree
tbl['w1mpro'] = tbl['w1mpro']*u.mag 
tbl['w1sigmpro'] = tbl['w1sigmpro']*u.mag
tbl['w2mpro'] = tbl['w2mpro']*u.mag
tbl['w2sigmpro'] = tbl['w2sigmpro']*u.mag
tbl['w3mpro'] = tbl['w3mpro']*u.mag
tbl['w3sigmpro'] = tbl['w3sigmpro']*u.mag
tbl['w1flux'] = tbl['w1flux']*u.mJy
tbl['w1sigflux'] = tbl['w1sigflux']*u.mJy
tbl['ML_FLUX_1'] = tbl['ML_FLUX_1']*u.erg/(u.cm**2*u.s)
tbl['ML_FLUX_ERR_1'] = tbl['ML_FLUX_ERR_1']*u.erg/(u.cm**2*u.s)
tbl['ML_FLUX_P4'] = tbl['ML_FLUX_P4']*u.erg/(u.cm**2*u.s)
tbl['ML_FLUX_ERR_P4'] = tbl['ML_FLUX_ERR_P4']*u.erg/(u.cm**2*u.s)
tbl['Radio flux'] = tbl['Radio flux']*u.mJy
tbl['Radio freq'] = tbl['Radio freq']*u.GHz
tbl['Alpha IRX'] = tbl['Alpha IRX']
tbl['Alpha RIR'] = tbl['Alpha RIR']
tbl['r_mag'] = tbl['r_mag']*u.mag
table_hdu = fits.BinTableHDU(tbl) 
header = table_hdu.header
header['EXTNAME'] = 'SOURCE_CATALOG'
header['AUTHOR'] = 'Cassie Metzger' 
header['DATE'] = '2025-04-03' 
header['WAVELENG'] = 'Multi'
header['DATASRC'] = 'eROSITA + WISE + Assorted radio catalogs'
header['DOI'] = '10.48550/arXiv.2501.12520' 
header['WAVELENG'] = 'Multi'
header['SELPRO'] = 'Sources have WISE magnitudes less than 14.3, 13.8, and 12.2 in the 3.4um, 4.6um, and 12um bands, respectively, fall within 5'' of an eRASS1 source, fall within 0.21 of the best fit line y = 0.84x -1.05, have alpha_IRX >= -1, are not extended in the X-ray wavelength, are not extragalactic, have a radio counterpart with 5'', and have alpha_RIR >= -0.43'
tbl.write('./catalogs/THC_catalog.fits', format = 'fits', overwrite = True)
print('Catalog saved as THC_catalog.fits')

Catalog saved as THC_catalog.fits
